# 앙상블 검색기(Ensemble Retriever)

`EnsembleRetriever`는 여러 retriever를 입력으로 받아 `get_relevant_documents()` 메서드의 결과를 앙상블하고, [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) 알고리즘을 기반으로 결과를 재순위화합니다.

서로 다른 알고리즘의 장점을 활용함으로써, `EnsembleRetriever`는 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

가장 일반적인 패턴은 `sparse retriever` (예: BM25)와 `dense retriever` (예: embedding similarity)를 결합하는 것인데, 이는 두 retriever의 장점이 상호 보완적이기 때문입니다. 이를 **"hybrid search"** 라고도 합니다.

Sparse retriever는 키워드를 기반으로 관련 문서를 찾는 데 효과적이며, dense retriever는 의미적 유사성을 기반으로 관련 문서를 찾는 데 효과적입니다.


In [4]:
%pip install -qU rank_bm25 deeplake > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [5]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [6]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

- 두 개의 문서 리스트(`doc_list_1`과 `doc_list_2`)를 정의합니다.
- `EnsembleRetriever`를 초기화하여 `BM25Retriever`와 `FAISS` 검색기를 결합합니다. 각 검색기의 가중치는 0.5로 설정됩니다.


In [47]:
# 비타민 별 섭취할 수 있는 음식 정보
doc_list_1 = [
    "비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.",
    "비타민B : 전곡물, 콩, 견과류, 육류 등 다양한 식품에서 찾을 수 있습니다.",
    "비타민C : 오렌지, 키위, 딸기, 브로콜리, 피망 등의 과일과 채소에 많이 들어 있습니다.",
    "비타민D : 연어, 참치, 버섯, 우유, 계란 노른자 등에 함유되어 있습니다.",
    "비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.",
]

# 비타민 별 효능 정보
doc_list_2 = [
    "비타민A : 시력과 피부 건강을 지원합니다.",
    "비타민B : 에너지 대사와 신경계 기능을 돕습니다.",
    "비타민C : 면역 체계를 강화하고 콜라겐 생성을 촉진합니다.",
    "비타민D : 뼈 건강과 면역 체계를 지원합니다.",
    "비타민E : 항산화 작용을 통해 세포를 보호합니다.",
]


# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_texts(
    # doc_list_1의 텍스트와 메타데이터를 사용하여 BM25Retriever를 초기화합니다.
    doc_list_1,
    metadatas=[{"source": 1}] * len(doc_list_1),
)
bm25_retriever.k = 1  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

embedding = OpenAIEmbeddings()  # OpenAI 임베딩을 사용합니다.
faiss_vectorstore = FAISS.from_texts(
    # doc_list_2의 텍스트와 임베딩, 메타데이터를 사용하여 FAISS 벡터 저장소를 초기화합니다.
    doc_list_2,
    embedding,
    metadatas=[{"source": 2}] * len(doc_list_2),
)
# FAISS 벡터 저장소를 사용하여 retriever를 생성하고, 검색 결과 개수를 1로 설정합니다.
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    # BM25Retriever와 FAISS retriever를 사용하여 EnsembleRetriever를 초기화하고, 각 retriever의 가중치를 0.5로 설정합니다.
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.6, 0.4],
    search_type="mmr",
)

`ensemble_retriever` 객체의 `get_relevant_documents()` 메서드를 호출하여 관련성 높은 문서를 검색합니다.


In [58]:
# 검색 결과 문서를 가져옵니다.
query = "비타민A 의 효능은?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.', metadata={'source': 1}), Document(page_content='비타민A : 시력과 피부 건강을 지원합니다.', metadata={'source': 2})]

[BM25 Retriever]
 [Document(page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.', metadata={'source': 1})]

[FAISS Retriever]
 [Document(page_content='비타민A : 시력과 피부 건강을 지원합니다.', metadata={'source': 2})]



In [59]:
# 검색 결과 문서를 가져옵니다.
query = "시력에 좋은 비타민은?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.', metadata={'source': 1}), Document(page_content='비타민A : 시력과 피부 건강을 지원합니다.', metadata={'source': 2})]

[BM25 Retriever]
 [Document(page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.', metadata={'source': 1})]

[FAISS Retriever]
 [Document(page_content='비타민A : 시력과 피부 건강을 지원합니다.', metadata={'source': 2})]



In [60]:
# 검색 결과 문서를 가져옵니다.
query = "비타민E 는 어떻게 섭취할 수 있나요?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.', metadata={'source': 1}), Document(page_content='비타민E : 항산화 작용을 통해 세포를 보호합니다.', metadata={'source': 2})]

[BM25 Retriever]
 [Document(page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.', metadata={'source': 1})]

[FAISS Retriever]
 [Document(page_content='비타민E : 항산화 작용을 통해 세포를 보호합니다.', metadata={'source': 2})]



## 런타임 설정

런타임에서도 retriever를 구성할 수 있습니다. 이를 위해서는 필드를 구성 가능한 것으로 표시해야 합니다.


- `ConfigurableField` 클래스를 `langchain_core.runnables` 모듈에서 임포트합니다.


In [10]:
from langchain_core.runnables import ConfigurableField

- `faiss_retriever` 객체를 생성하며, `faiss_vectorstore`의 `as_retriever` 메서드를 사용합니다.
  - `search_kwargs` 매개변수를 통해 검색 시 반환할 결과의 개수(k)를 2로 설정합니다.
- `configurable_fields` 메서드를 사용하여 `faiss_retriever`의 설정 가능한 필드를 정의합니다.
  - `search_kwargs` 필드를 `ConfigurableField` 객체로 정의합니다.
  - 필드의 ID는 "search_kwargs_faiss"로 설정합니다.
  - 필드의 이름은 "Search Kwargs"로 설정합니다.
  - 필드에 대한 설명을 "The search kwargs to use"로 설정합니다.


In [11]:
faiss_retriever = faiss_vectorstore.as_retriever(
    # 검색 시 반환할 결과의 개수를 설정합니다.
    search_kwargs={"k": 2}
).configurable_fields(
    search_kwargs=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정합니다.
        id="search_kwargs_faiss",
        # 검색 매개변수의 이름을 설정합니다.
        name="Search Kwargs",
        # 검색 매개변수에 대한 설명을 작성합니다.
        description="The search kwargs to use",
    )
)

`EnsembleRetriever`를 사용하여 두 개의 검색기를 앙상블로 결합하는 코드입니다.

- `bm25_retriever`와 `faiss_retriever`를 리스트로 전달하여 앙상블에 포함시킵니다.
- `weights` 파라미터를 통해 각 검색기의 가중치를 [0.5, 0.5]로 설정합니다. 이는 두 검색기의 결과를 동일한 비율로 반영함을 의미합니다.


In [12]:
ensemble_retriever = EnsembleRetriever(
    # 리트리버 목록을 설정합니다. 여기서는 bm25_retriever와 faiss_retriever를 사용합니다.
    retrievers=[bm25_retriever, faiss_retriever],
    # 각 리트리버의 가중치를 설정합니다. 여기서는 두 리트리버에 동일한 가중치 0.5를 부여합니다.
    weights=[0.5, 0.5],
)

`ensemble_retriever`를 사용하여 "apples"라는 질의어에 대한 문서를 검색합니다.

- 검색 시 `config` 매개변수를 통해 검색 설정을 지정합니다.
  - `search_kwargs_faiss` 옵션의 `k` 값을 1로 설정하여 가장 관련성이 높은 문서 1개를 반환하도록 합니다.

FAISS retriever에서 하나의 source만 반환되는 것에 주목해 주세요. 이는 실행 시점에 관련 설정을 전달하기 때문입니다.


In [15]:
# 검색 설정을 지정합니다. Faiss 검색에서 k=1로 설정하여 가장 유사한 문서 1개를 반환하도록 합니다.
config = {"configurable": {"search_kwargs_faiss": {"k": 1}}}
# 앙상블 검색기를 사용하여 "apples"에 대한 검색을 수행하고, 설정된 config를 적용합니다. 검색 결과는 docs 변수에 저장됩니다.
docs = ensemble_retriever.invoke("apples", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(page_content='You like apples', metadata={'source': 2}),
 Document(page_content='I like apples', metadata={'source': 1}),
 Document(page_content='Apples and oranges are fruits', metadata={'source': 1})]